# Tutorial 5 - Symbolic Pursuit

In this tutorial we we create a symbolic pursuit explainer object and use it to get an explanation of the predictive model. The explainer is then saved to disk and can be given to someone else to view in the [Interpretability Suite App](https://vanderschaarlab-demo-interpretabi-interpretability-suite-1uteyn.streamlit.app/).

We will be explaining the predictions of a multilayer perceptron regressor provided by sci-kit learn that is trained on the diabetes dataset also from sci-kit learn.

### Import the relevant modules

In [ ]:
# IMPORTS

# Third Party
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

# Interpretability
from interpretability.interpretability_models import symbolic_pursuit_explainer
from interpretability.interpretability_models.utils import io

### Load the data 
Load the data and split it into the different sets for training the mlp, fitting the explainer and testing the explainer.

In [ ]:

X, y = load_diabetes(return_X_y=True, as_frame=True)

feature_names = X.columns.to_list()
X_mlp_train, X_test, y_mlp_train, y_test = train_test_split(X, y, test_size=0.5)
X_expl_train, X_explain_test, y_expl_train, y_explain_test = train_test_split(X_test, y_test, test_size=0.2)

X_train, X_test, y_train, y_test = (
    X_mlp_train.to_numpy(),
    y_mlp_train.to_numpy(),
    X_expl_train.to_numpy(),
    y_expl_train.to_numpy(),
    X_explain_test.to_numpy(),
    y_explain_test.to_numpy(),
)

### Train the model

We will train our own model here using the sklearn library here. We simply have to initialize it and fit it, before it is ready to pass to the SymbolicPursuitExplainer in the next step.

In [ ]:
## Load the model

model = MLPRegressor()
model.fit(X_mlp_train, y_mlp_train)

### Initialize and fit Symbolic Pursuit Explainer
Initialize the explainer object by passing the models predict function, and the data to explain. The fit step can take some considerable time, this can be reduced by reducing the `patience` argument or increasing the `loss_tol` argument.

In [ ]:

my_explainer = symbolic_pursuit_explainer.SymbolicPursuitExplainer(
    model.predict, X_expl_train, feature_names=feature_names, loss_tol=0.01, patience=5
)
my_explainer.fit()

### Measure fit quality
This prints the Mean Squared Error for both the mlp predictive model and the learned symbolic model

In [ ]:
my_explainer.measure_fit_quality(X_explain_test, y_explain_test)

### Get the explanation
Get the explanation of the model in terms of its symbolic expression and projections of the predictive model.

In [ ]:
explanation = my_explainer.explain()

### Show the symbolic expression and projections

In [ ]:

my_explainer.summary_plot()

### Save the explainer to file
This file can now be uploaded to the Interpretability Suite app (TODO: add link). This provides a non-programtic interface with which to view the various explanations, allowing you to send the explainer to a colleague who is less fluent in python.

In [ ]:

io.save_explainer(
    my_explainer, "my_new_diabetes_pytorch_mlp_symbolic_pursuit_explainer.p"
)